# Fuzzy duplicate detection
fingerprinting

Two strings can easily be compared using https://github.com/seatgeek/fuzzywuzzy

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

In [2]:
d = pd.DataFrame({'one': ['fuzz', 'wuzz'], 'two': ['fizz', 'woo']})

d.apply(lambda s: fuzz.partial_ratio(s['one'], s['two']), axis=1)

0    75
1    33
dtype: int64

But what if I want to perform such a operation with each row of a dataframe vs. all the other rows e.g. find duplicates?

In [3]:
d = pd.DataFrame({'id': ['1', '2', '3'], 'email': ['first@email.com', '1@email.com', 'iTrickYouAndStealIBAN1sBank@other.com'], 'bank': ['IBAN1', 'IBAN2', 'IBAN1'], 'name': ['name1', 'name1', 'name3'], 'date': ['2016-01-01', '2016-01-02', '2016-01-02']})
d

,bank,date,email,id,name
0,IBAN1,2016-01-01,first@email.com,1,name1
1,IBAN2,2016-01-02,1@email.com,2,name1
2,IBAN1,2016-01-02,iTrickYouAndStealIBAN1sBank@other.com,3,name3


Get each row as a string. Or would you recommend to calculate the distance per field and add them up?

In [4]:
x = d.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
vals = pd.DataFrame([','.join(ele.split()) for ele in x])
vals

,0
0,"IBAN1,2016-01-01,first@email.com,1,name1"
1,"IBAN2,2016-01-02,1@email.com,2,name1"
2,"IBAN1,2016-01-02,iTrickYouAndStealIBAN1sBank@o..."


The numbers here do not work out e.g. index 2 / id3 should be a duplicate with 1. Ideally certain columns e.g. bank would be weighted like http://stackoverflow.com/questions/19964546/python-pandas-fuzzy-merge-match-with-duplicates

In [63]:
#vals.apply(lambda s: fuzz.partial_ratio(s, vals), axis=1)

Here, at least the numers work correctly

In [64]:
#vals.apply(lambda s: fuzz.token_set_ratio(s, vals), axis=1)

### Question 
How to find duplicates of one column vs. all the other ones without a gigantic for loop of converting row_i toString() and then comparing it to all the other ones?

Use the date column and add and additional column to `d` which describes *daysSinceLastPurchase*, which e.g. for *id 3* should be `today - 2016-01-01` because the "real last purches" occured just a day before

To fill the min-hash I need to loop over all rows and all fields n*k * n to add all the sets to the LSH.


# after the initial question -> BK tree
https://gist.github.com/nibogd/94363e93f4e0256b4665eb743dbfa211

# try clustering

In [57]:
#from fuzzywuzzy import fuzz
#import numpy as np
#from sklearn.cluster import dbscan

#def metric(x, y):
#    i, j = int(x[0]), int(y[0])     
#    return 1 - fuzz.ratio((vals[i], vals[j]) / 100.)

#X = np.arange(len(vals)).reshape(-1, 1)
#dbscan(X, metric=metric, eps=5, min_samples=2)  

# try lsh / min-hash
how to efficiently loop here

In [83]:
# create as many min-hash's as needed
allCustomers = []
lsh = MinHashLSH(threshold=0.5, num_perm=128)

for index, row in d.iterrows():
    
    print("index, ", index)
    currCust = MinHash(num_perm=128)
    for column in d.columns:
        # add all the columns to min-hash
        
        # take everything, TODO try selected attributes
        currCust.update(d[column].encode('utf8'))
        
    # add filled min-hash to LSH
    allCustomers.append(currCust)
    lsh.insert(index, currCust)
    

index,  0


AttributeError: 'Series' object has no attribute 'encode'

In [56]:
from datasketch import MinHash, MinHashLSH

data1 = ['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'datasets']
data2 = ['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'documents']
data3 = ['minhash', 'is', 'probability', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'documents']

# Create MinHash objects
m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in data1:
    m1.update(d.encode('utf8'))
for d in data2:
    m2.update(d.encode('utf8'))
for d in data3:
    m3.update(d.encode('utf8'))

# Create an MinHashLSH index optimized for Jaccard threshold 0.5,
# that accepts MinHash objects with 128 permutations functions
lsh = MinHashLSH(threshold=0.5, num_perm=128)

# Insert m2 and m3 into the index
lsh.insert("m2", m2)
lsh.insert("m3", m3)

# Check for membership using the key
print("m2" in lsh)
print("m3" in lsh)

# Using m1 as the query, retrieve the keys of the qualifying datasets
result = lsh.query(m1)
print("Candidates with Jaccard similarity > 0.5", result)

# Remove key from lsh
lsh.remove("m2")

True
True
Candidates with Jaccard similarity > 0.5 ['m2', 'm3']
